In [1]:
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.utils.data_utils import get_file
from sklearn.decomposition import PCA
import numpy as np
import scipy.io
import cv2
from tqdm import tqdm
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


In [2]:
# create a dictionary of annotations structures
# hashed by file name
# length of value is number of persons annotated
# [['person'],['gender'],['annotations_categories'],['annotations_continuous'],['gender'],['age']]
def create_emotic_dict(data, test= False ): 
    # set variables based on testing/training dataset 
    # using combined avg on test dataset because 
    # multiple annotators used 
    if test: 
        categories = 'combined_categories'
        continuous  = 'combined_continuous'
    else: 
        categories = 'annotations_categories'
        continuous  = 'annotations_continuous'
    
    
#     test_image = 'framesdb/images/frame_ghkq7yp0itqz0kn7.jpg'
    emotic_annotations = {} 
    
    print ("Creating body boxes")
    for i in range(len(data['filename'][0])): 
        file = data['filename'][0][i][0]
        # if more than one person is the focus of the image 
        for j in range(len(data['person'][0][i][0])):      
            file_path = str(data['folder'][0][i][0]) + "/" + file
            
            if file_path in emotic_annotations: 
                emotic_annotations[file_path] = np.append( emotic_annotations[file_path], 
                                                # (x1, y1, x2, y2)
                                                [[tuple([int(i) for i in np.array(data['person'][0][i][0][j]['body_bbox'][0]).tolist()]),
                                                  np.array(np.array(np.array(data['person'][0][i][0][j][categories]).tolist()).tolist()).flatten().tolist(),      
                                                  np.array(np.array(data['person'][0][i][0][j][continuous]).tolist()).flatten().tolist(),
                                                  np.array(data['person'][0][i][0][j]['gender']).tolist(),
                                                  np.array(data['person'][0][i][0][j]['age']).tolist()]], 
                                                  axis = 0 )

            else: 
                emotic_annotations[file_path]= np.array([[tuple([int(i) for i in np.array(data['person'][0][i][0][j]['body_bbox'][0]).tolist()]),
                                               np.array(np.array(np.array(data['person'][0][i][0][j][categories]).tolist()).tolist()).flatten().tolist(),                             
                                               np.array(np.array(data['person'][0][i][0][j][continuous]).tolist()).flatten().tolist(),
                                               np.array(data['person'][0][i][0][j]['gender']).tolist(),
                                               np.array(data['person'][0][i][0][j]['age']).tolist()]])
    
    return emotic_annotations

emotic = scipy.io.loadmat('EMOTIC/annotations/Annotations.mat')
train = create_emotic_dict(emotic['train'])
test = create_emotic_dict(emotic['test'], True)


Creating body boxes
Creating body boxes


In [3]:
 class PersonFeatures():
    def __init__(self, train, test):
        print ("LETS DO THIS ")
        # pull out body boxes
        self.train = train
        self.test = test
        
        print ("XTRAIN")
        X_train = self.extract_bodies(self.train)
        print ("XTEST")
        X_test = self.extract_bodies(self.test, True)

        # Compute a PCA 
        n_components = 100
        print ("PCA")
        pca = PCA(n_components=n_components, whiten=True).fit(X_train)

        # apply PCA transformation
        X_train_pca = pca.transform(X_train)
        X_test_pca = pca.transform(X_test)          
 
        self.train_feats = self.extract_features(X_train_pca)
        self.test_feats = self.extract_features(X_test_pca)


    def extract_bodies(self, images, test = False):
        print ("BODIES")

        dir_path = "EMOTIC/emotic/"
        out_path = "EMOTIC/emotic/cropped/train/"
        if test: 
            out_path = "EMOTIC/emotic/cropped/test/"

#         test_image = 'framesdb/images/frame_ghkq7yp0itqz0kn7.jpg'
        # crop images
        cropped_images = []
        # load in training images body boxes
        for k, v in tqdm(images.items()):
            for b in v[:, 0]:
                # read image for cropping
                img = cv2.imread(dir_path+k)
    #             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#                 cv2.imshow("image", img)
                cropped_img = img[ b[1]:b[3], b[0]:b[2] ]
    #                 cv2.imshow("cropped", cropped_img)
#                     cv2.waitKey(0)
    #                 cv2.imwrite(out_path + k, cropped_img)

                cropped_images.append(cropped_img)
    
        return cropped_images

        
    def extract_features(self, images):
        print ("FEATURES")
        
        face_model = VGGFace(model='resnet50', include_top=False)

        # list of pre-processed images 
        imgs = [] 
        for i in images: 
            # load image setting the image size to 224 x 224
            target_size = (224, 224)
            img = cv2.imread(i,1)
            img = cv2.resize(img, target_size)
#             cv2.imshow('image',img)
#             cv2.waitKey(0)

            # convert image to numpy array
            x = image.img_to_array(img)
            # the image is now in an array of shape (3, 224, 224) 
            # need to expand it to (1, 3, 224, 224) as preprocess takes in a list
            x = np.expand_dims(x, axis=0)
            # ensure proper input format for model 
            x = preprocess_input(x)
            
            # extract the features
            imgs.append(x) 
            
        imgs = np.concatenate(imgs, axis=0)
        img_feats = face_model.predict(imgs)
        
        return img_feats


In [ ]:
p = PersonFeatures(train, test)

print(p.train_feats)
print(p.test_feats)

LETS DO THIS 
XTRAIN
BODIES
XTEST
BODIES
